# CONTEXTE
===================


## _Sujet_

Vous travaillez pour une société de vente de paddle. Cette société est implantée en région Centre-Val-de-Loire dans les villes d’Orléans, Blois, Tours et Bourges. Vos missions au sein de la DSI portent sur l’analyse de données et le développement d’applications permettant de traiter des données.

## Projet

Le service RH veut mettre en place une application de gestion du personnel de la société puis réaliser
quelques statistiques sur les salariés. L’enjeu est d’améliorer la gestion actuelles des personnels.
Cette application doit permettre aux salariés du service RH de saisir les données sur les personnels
actuels et futurs et de collecter les statistiques suivantes :
1. Combien de salariés comprend la société ?
2. Où sont-ils localisés géographiquement au niveau professionnel ?
3. Quel est l’âge moyen et médian des salariés ?
4. Quel est l’étendue des âges ?
5. Quel est le salarié le plus ancien de la société en termes d’ancienneté ?
6. Quelle est la répartition entre les hommes et les femmes ?
7. Y-a-t-il des doublons ?
8. Afficher la liste des salariés classés par ordre alphabétique.

## Proposition

Voir la documentation associée Gestion_personnel_documentation.docx [lien](Gestion_personnel_documentation.docx)

![Image](image_paddle.jpg "icon")


# FONCTIONNALITES ADMINISTRATEUR
==============================================================

## Import des bibliothèques

- SQLite permet de gérer une base de données avec des instructions SQL


In [2]:
import sqlite3


In [3]:
# dir(sqlite3)

## Connexion

- Connecter la base 
- Connecter le curseur

In [4]:
def connecter():
    """
    :name : connecter
    :param : 
    :return : curseur et connexion nécessaires à d'autres fonctions
    :desc : se connecter à la base et activer le curseur
    """

    try:
        connexion = sqlite3.connect("Gestion_personnel.db")
        curseur = connexion.cursor()
        return curseur, connexion

    except sqlite3.Error as e:
        print(e, "\n\n=====> La connexion ne s'est pas établie")
    


## Ajout des tables dans la base

- Création d'une variable pour chaque table
- Script SQL: CREATE TABLE
- Exécution du script

In [5]:
def creer_employes(curseur):
    
    """
    :name : creer_employes
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une table dans la base de données.
    """
    
    
    try:
        table_employes = """
                            CREATE TABLE IF NOT EXISTS "employes" (
                            "id_em"	INTEGER NOT NULL UNIQUE,
                            "nom"	VARCHAR(40) NOT NULL,
                            "prenom"	VARCHAR(40) NOT NULL,
                            "date_naissance"	DATE NOT NULL,
                            "date_entree"	DATE NOT NULL,
                            "date_sortie"	DATE,
                            "id_ge"	BYTE NOT NULL,
                            "id_si"	SMALLINT NOT NULL,
                            PRIMARY KEY("id_em" AUTOINCREMENT),
                            FOREIGN KEY("id_si") REFERENCES "sites"("id_si"),
                            FOREIGN KEY("id_ge") REFERENCES "genres"("id_ge")
        )
                            """



        curseur.execute(table_employes)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> La table employes n'a pas été créée")

In [6]:
def creer_sites(curseur):
    """
    :name : creer_sites
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une table dans la base de données.
    """

    try:
        table_sites = """
						CREATE TABLE IF NOT EXISTS "sites" (
						"id_si"	INTEGER NOT NULL UNIQUE,
						"site"	VARCHAR(40) NOT NULL UNIQUE,
						UNIQUE("site"),
						PRIMARY KEY("id_si" AUTOINCREMENT)
	)
						"""

        curseur.execute(table_sites)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> La table sites n'a pas été créée")

In [7]:
def creer_genres(curseur):

    """
    :name : creer_genres
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une table dans la base de données.
    """
    
    try:
        table_genres = """
                            CREATE TABLE IF NOT EXISTS "genres" (
                            "id_ge"	INTEGER NOT NULL UNIQUE,
                            "genre"	VARCHAR(5) NOT NULL UNIQUE,
                            PRIMARY KEY("id_ge" AUTOINCREMENT),
                            UNIQUE("genre")
        )
                            """



        curseur.execute(table_genres)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> La table genres n'a pas été créée")

## Les triggers

- Création d'une variable pour chaque trigger
- Script SQL: CREATE TRIGGER
- Exécution du script


1er trigger pour vérifier que la date d'entrée est inférieure à la date de naissance + 16 ans

In [8]:
def verifier_date_entree(curseur):

    """
    :name : verifier_date_entree
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une condition empêchant l'insertion d'un enregistrement dans la base de données. 
            la date d'entrée doit être inférieure à la date de naissance + 16 ans
    """ 
    
    try:
        date_entree_valide = """
                    CREATE TRIGGER IF NOT EXISTS verif_date_entree
                    BEFORE INSERT ON employes
                    FOR EACH ROW
                        WHEN  julianday(NEW.date_entree) < (julianday(NEW.date_naissance)+(16*365.25))
                        BEGIN
                            SELECT RAISE(ABORT, "Votre employé ne peut pas entrer s'il a moins de 16 ans");
                        END;
                    """
        curseur.execute(date_entree_valide)
    
    except sqlite3.Error as e:
        print(e, "\n\n=====> Le trigger verif_date_entree n'a pas été créé")
        
        


2ème trigger pour vérifier que la date de sortie est supérieure à la date d'entrée

In [9]:
def verifier_date_sortie(curseur):

    """
    :name : verifier_date_sortie
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une condition empêchant l'insertion d'un enregistrement dans la base de données. 
            la date de sortie doit être supérieur à la date d'entrée
    """ 
    try:
        date_sortie_valide = """
                    CREATE TRIGGER IF NOT EXISTS verif_date_sortie
                    BEFORE UPDATE ON employes
                    FOR EACH ROW
                        WHEN julianday(NEW.date_sortie) <= julianday(NEW.date_entree)
                        BEGIN
                            SELECT RAISE(ABORT, "Votre employé doit sortir au moins un jour après son entrée.");
                        END;
                    """
        curseur.execute(date_sortie_valide)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> Le trigger verif_date_sortie n'a pas été créé")

3ème trigger pour vérifier que le code du genre existe

In [10]:
def foreign_key_contrainst_genre(curseur):

    """
    :name : foreign_key_contrainst_genre
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une condition empêchant l'insertion d'un enregistrement dans la base de données. 
            le code du genre doit être compris dans la table genres
    """ 
    
    try:
        genre_valide = """
                    CREATE TRIGGER IF NOT EXISTS verif_genre
                    BEFORE INSERT ON employes
                    FOR EACH ROW
                        WHEN NEW.id_ge not in (SELECT id_ge FROM genres)
                        BEGIN
                            SELECT RAISE(ABORT, "Le code du genre doit correspondre à l'un de ceux dans la liste qui précédait.");
                        END;
                    """
        curseur.execute(genre_valide)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> Le trigger verif_genre n'a pas été créé")
        


4ème trigger pour vérifier que le code du site existe

In [11]:
def foreign_key_contrainst_site(curseur):
    

    """
    :name : foreign_key_contrainst_genre
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : création d'une condition empêchant l'insertion d'un enregistrement dans la base de données. 
            le code du genre doit être compris dans la table genres
    """ 
    
    try:
        site_valide = """
                    CREATE TRIGGER IF NOT EXISTS verif_site
                    BEFORE INSERT ON employes
                    FOR EACH ROW
                        WHEN NEW.id_si > (SELECT count(*) FROM sites)
                        BEGIN
                            SELECT RAISE(ABORT, "Le code du site doit correspondre à l'un de ceux dans la liste qui précédait.");
                        END;
                    """
        curseur.execute(site_valide)
        
    except sqlite3.Error as e:
        print(e, "\n\n=====> Le trigger verif_site n'a pas été créé")
            
    

## Vérification 

- Afficher le contenu de la base: sqlite_schema

In [12]:
def verifier(curseur):
    
    """
    :name : verifier
    :param : curseur retourné par la fonction connecter()
    :return : 
    :desc : afficher les tables contenues dans la base de données
    """ 
    
    
    try:
        curseur.execute("SELECT name FROM sqlite_schema WHERE type='table'")
        verification = curseur.fetchall()
        return verification

    except sqlite3.Error as e:
        print(e, "\n\n=====> L'affichage de la liste des tables a échoué")

## Deconnexion

- fermer le curseur
- fermer la base

In [13]:
def deconnecter(curseur,connexion):
    
    """
    :name : deconnecter
    :param : curseur et connexion retournés par la fonction connecter()
    :return : 
    :desc : désactiver le curseur et se déconnecter de la base
    """ 
    
    try:
        curseur.close()
        connexion.close()
    except sqlite3.Error as e:
        print(e, "\n\n=====> La déconnexion ne s'est pas réalisée")

## Charger les tables avant "golive"

- Script SQL: INSERT INTO 
- Exécution du script

### Migration de la table des employes

Préalablement:
- Si une liste existe, l'exporter au format CSV avec 7 colonnes:
    - Nom => obligatoire
    - Prénom => obligatoire
    - Date de naissance => obligatoire
    - Date d'entrée => obligatoire
    - Date de sortie => vide si non sorti
    - Site => obligatoire + à convertir avec l'id_si de la table sites 
    - Genre => obligatoire + à convertir avec l'id_ge de la table genres 
- Maintenir la 1ère ligne pour les titres
- Nommer le fichier liste_employes.csv

Ici:
- charger le fichier
- pour chaque ligne
    - construire la requête avec un INSERT INTO
    - avec la fonction se_connecter(), exécuter la requête


In [14]:
def charger_employes(curseur,connexion):
    return "*** Fonctionnalité pas encore disponible ***"

### Pré-charger les sites par défaut

In [15]:
def charger_sites(curseur,connexion):

    """
    :name : charger_sites
    :param : curseur et connexion retournés par la fonction connecter()
    :return : 
    :desc : charger les 4 sites actuels
    """ 
        
    try:
    
        curseur.execute(""" INSERT INTO sites (site) VALUES ("Orléans") """)
        curseur.execute(""" INSERT INTO sites (site) VALUES ("Blois") """)
        curseur.execute(""" INSERT INTO sites (site) VALUES ("Tours") """)
        curseur.execute(""" INSERT INTO sites (site) VALUES ("Bourges") """)


        connexion.commit()
        curseur.execute("SELECT * FROM sites")
        verification = curseur.fetchall()
        return verification
    
    except sqlite3.Error as e:
        print(e, "\n\n=====> La table sites n'a pas été chargée")

    

### Pré-charger les genres par défaut

In [16]:
def charger_genres(curseur,connexion):
    
    """
    :name : charger_genres
    :param : curseur et connexion retournés par la fonction connecter()
    :return : 
    :desc : charger les 4 sites actuels
    """ 
    
    try:
        curseur.execute(""" INSERT INTO genres (genre) VALUES ("Femme") """)
        curseur.execute(""" INSERT INTO genres (genre) VALUES ("Homme") """ )

        connexion.commit()
        curseur.execute("SELECT * FROM genres")
        verification = curseur.fetchall()
        return verification
    
    except sqlite3.Error as e:
        print(e, "\n\n=====> La table genres n'a pas été chargée")

# FONCTIONNALITES UTILISATEUR
====================================================

## Se connecter à la base

- chaque requête sera traitée seule
- Se connecter et activer le curseur
- Exécuter + commiter
- Fermer le curseur et la base


In [17]:
def se_connecter(requete):
    """
    :name : se_connecter
    :param : requete permettant de passer une instruction en sql
    :return : 
    :desc : la base va se connecter et se déconnecter à chaque passage de requête
    """

    try:
    
        connexion = sqlite3.connect("Gestion_personnel.db")
        curseur = connexion.cursor()
        curseur.execute(requete)
        connexion.commit()
        print("=====> Instruction passée") 
        curseur.close()
        connexion.close()
    
    except sqlite3.Error as e:
        print(e, "\n\n=====> Une action ne s'est pas réalisée (connexion, exécution ou déconnexion)")




## Travailler sur une table

- Sélectionner la table 
- si ok, sélectionner l'action: afficher, saisir, modifier, supprimer
- demander la confirmation
- si ok, conditions selon le mix choix de la table + choix de l'action

In [18]:
def travailler_tables():

    """
    :name : travailler_tables
    :param : 
    :return : 
    :desc : gérer la navigation de l'utilisateur pour travailler sur une table 
    """ 
    
    print("*************************************************************************")
    print("*                    TRAVAIL SUR LES TABLES                             *")
    print("*                     Gestion_personnel.db                              *")
    print("*                                                                       *")
    print("*************************************************************************")
    
    
    afficher_liste = ["employes","genres","sites"]
    choix_table = menu_selection(afficher_liste)


    
    if choix_table < len(afficher_liste) :
        
        afficher_liste2 = ['afficher','saisir','modifier','supprimer']
        choix_action = menu_selection(afficher_liste2)


        print(f"Vous allez travailler sur la table {afficher_liste[choix_table]} pour {afficher_liste2[choix_action]} ")
        confirmation = confirmer()

        if confirmation:   

            if choix_table == 0 and choix_action == 1:
                continuer = "o"
                while continuer =="o":
                    se_connecter(saisir_employe()) 
                    continuer = input("Voulez-vous saisir une autre fiche? (o/n)")

            elif choix_table == 0 and choix_action == 2:
                continuer = "o"
                while continuer =="o":
                    resultat = modifier_employe()
                    if resultat is not None:
                        se_connecter(resultat) 
                    continuer = input("Voulez-vous modifier une autre fiche? (o/n)")
                    
                        

            
            elif choix_table == 0 and choix_action == 0:
                afficher_table("employes")
                
            elif choix_table == 1 and choix_action == 0:
                afficher_table("genres")
            
            elif choix_table == 2 and choix_action == 0:
                afficher_table("sites")
            
            elif choix_table == 0 and choix_action == 3:
                resultat = supprimer_employe()
                if resultat is not None:
                    continuer = "o"
                    while continuer =="o":
                        se_connecter(resultat) 
                        continuer = input("Voulez-vous suppimer une autre fiche? (o/n)")
            
            else:
                print("*** Fonctionnalité pas encore disponible ***")


        print("\n\n=====> Terminé: retour au menu utilisateur")
        menu_utilisateur()
            
#travailler_tables()           

Fonction permettant de sécuriser la saisie ou la modification d'une date:
- tant que la date n'est pas valide, saisir une bonne date
- saisie au format français 'jj/mm/aaaa'
- vérification que 
    - l'année, le mois et le jour sont possible pour valider la date
    - le nombre de caractère est suffisant



In [19]:
def saisir_date():

    """
    :name : saisir_date
    :param : 
    :return : la date dans le format par défaut d'une base SQL 'yyyy-mm-dd'
    :desc : demander la saisie d'une date valide dans le format français puis la convertir dans le format de la base de données
    """ 
      
    valide = 'n'   
    
    
    while valide =="n":
        
        votre_date = input("Veuillez saisir une date valide (jj/mm/aaaa): ")

        jour, mois, annee = votre_date.split("/")
    
        bonne_annee = 1900 < int(annee) < 2025
    
        bon_mois = (0 < int(mois) <= 12) and len(mois)==2

        if (2020 - int(annee))%4==0:
            nbJour_mois =[31,29,31,30,31,30,31,31,30,31,30,31]
        else:
            nbJour_mois =[31,28,31,30,31,30,31,31,30,31,30,31]
        bon_jour = (0 < int(jour) <= nbJour_mois[int(mois)-1])  and len(jour)==2

        votre_date = f'{annee}-{mois}-{jour}'
        if bonne_annee and bon_mois and bon_jour :
            print(f"Votre date est valide et sera saisie : {votre_date} ")
            valide = "o"
        else:
            print("\n=====> Votre date n'existe pas et ne sera pas saisie : ")



    return votre_date


### Saisir la fiche d'un employé

- S'agissant d'une saisie manuelle, la date de sortie n'est pas proposé ici et restera nulle
- Script SQL: INSERT INTO

In [20]:
    
def saisir_employe():

    """
    :name : saisir_employe
    :param : 
    :return : l' argument de la fonction se_connecter
    :desc : insertion d'un nouvel employé, l'id est auto-incrémenter, la date_sortie n'est pas prévue ici.
    """

    nom = input("Quel est son nom de famille? ")
    
    prenom = input("Quel est son prénom? ")
    
    print("Vous allez saisir sa date de naissance")
    date_naissance = saisir_date()
    
    print("Vous allez saisir sa date d'entrée")
    date_entree = saisir_date()
    
    
    afficher_table("genres")
    genre = int(input("Saisir le code du genre: "))
    
    afficher_table("sites")                  
    site = int(input("Saisir le code du site: "))
            
    reponse ="'" + "','".join([nom, prenom,date_naissance, date_entree, str(genre), str(site)]) + "'"

    requete = f"INSERT INTO employes (nom, prenom, date_naissance, date_entree,id_ge,id_si) VALUES ({reponse})"
    print(requete)
    return requete

#se_connecter(saisir_employe())

### Modifier la fiche d'un employé

En particulier pour saisir la date de sortie

- commencer par chercher un employé par son nom 
- si plusieurs employés, demander quel id?


In [21]:
def rechercher_employe():
    
    """
    :name : rechercher_employe
    :param : 
    :return : le nombre de résultat et le dernier résultat
    :desc : rechercher le nom d'un employé en tenant compte de la casse. 
            cette fonction est utile aux fonctions modifier_employe et supprimer_employe
    """
    
    
    try:
        curseur, connexion = connecter()


        nom_employe = input("Quel nom cherchez-vous? ")

        requete = f"SELECT *  FROM employes WHERE nom = '{nom_employe}' "
        resultats = curseur.execute(requete)

        
        nb_res = 0
        

        for resultat in resultats:
            print(f" {resultat}\t *** id_em = {resultat[0]} ***")
            nb_res += 1
        print(f"{nb_res} employé(s) porte le nom {nom_employe}")

        deconnecter(curseur,connexion)
        
        if nb_res > 0:
            
            return nb_res, resultat[0]
        
    
    except sqlite3.Error as e:
        print(e, "\n\n=====> La recherche de votre employé a échoué.")


        

In [22]:
def afficher_employe(id_em):
    
    """
    :name : afficher_employe
    :param : identifiant de l'employé
    :return : la fiche de l'employé
    :desc : afficher simplement la fiche de l'employé. 
            cette fonction est utile aux fonctions modifier_employe et supprimer_employe
    """
    
    try:
        curseur, connexion = connecter()
        
        requete = f"SELECT *  FROM employes WHERE id_em = '{id_em}'  "
        resultat = curseur.execute(requete)

        fiche_employe = []

        for res in resultat:
            fiche_employe.append(res)
            # print(fiche_employe)

        deconnecter(curseur,connexion)

        
        return fiche_employe


    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de votre employé a échoué.")


In [23]:
def colonne_table():
    
    """
    :name : colonne_table
    :param : 
    :return : la liste des noms de colonne dans la table employes
    :desc : créer une liste pour être afficher dans la fonction modifier_employe
    """
    
    
    try:
        curseur, connexion = connecter()


        requete = "PRAGMA table_info (employes);  "
        resultat = curseur.execute(requete)

        fiche_colonne = []

        for res in resultat:
            fiche_colonne.append(res)
            # print(fiche_employe)

        deconnecter(curseur,connexion)


        return fiche_colonne

    except sqlite3.Error as e:
        print(e, "\n\n=====> La création de la liste des colonnes a échoué")

Pour modifier la fiche d'un employé:
- commencer par la rechercher en fonction de son nom
- si un résultat est retourné alors:
    - si plus d'un résultat, demander son choix à l'utilisateur
    - afficher la fiche
    - demander la confirmation
- si l'action a été confirmée alors:
    - créer la liste qui accueillera les valeurs de la nouvelle fiche
    - pour chaque colonne (sauf la 1ère contenant l'id), présenter son nom et son contenu
    - demander à l'utilisateur de confirmer le contenu
    - si non, alors: lui demander le nouveau contenu
    - splitter la nouvelle fiche pour facilité le formattage
    - script SQL: UPDATE (les nouvelles valeurs doivent être des strings)

In [24]:
def modifier_employe():
    
    
    """
    :name : modifier_employe
    :param : 
    :return : l' argument de la fonction se_connecter
    :desc : modifier la fiche d'un employé après l'avoir l'avoir cherchée et affichée
    """
    
    
    resultat = rechercher_employe()
    
    
    if resultat is not None:
           
        nb_res, last_res = resultat
        
        if nb_res != 1:
            last_res = input("Saisissez l'identifiant *** id_em *** de l'employé à modifier: ")


        fiche_employe = afficher_employe(last_res)

        print(f"=====> Vous allez travailler sur la fiche {fiche_employe} ")
        confirmation = confirmer()


        if confirmation:
            fiche_colonne = colonne_table()
            nouvelle_fiche_employe =[]

            for num,fiche in enumerate (fiche_colonne):
                if num == 0:
                    pass
                else:
                    print(f"Le champs {fiche_colonne[num][1]} donne {fiche_employe[0][num]}")
                    confirmation = confirmer()
                    if confirmation:
                        nouvelle_fiche_employe.append(fiche_employe[0][num])
                    else:
                        if "date" in fiche_colonne[num][1]:
                            nouvelle_fiche_employe.append(saisir_date())
                        else:
                            nouvelle_fiche_employe.append(input("Par quoi voulez-vous le remplacer? "))


            nv_nom,nv_prenom,nv_date_naissance,nv_date_entree,nv_date_sortie,nv_genre,nv_site = nouvelle_fiche_employe

            requete = f"UPDATE employes SET nom = '{nv_nom}', \
                                            prenom = '{nv_prenom}', \
                                            date_naissance = '{nv_date_naissance}', \
                                            date_entree = '{nv_date_entree}', \
                                            date_sortie = '{nv_date_sortie}', \
                                            id_ge = '{str(nv_genre)}', \
                                            id_si = '{str(nv_site)}' \
                                        WHERE id_em = '{last_res}' "




            print(requete)
            return requete
        
        
        
        
# modifier_employe()


Pour supprimer la fiche d'un employé:
- commencer par la rechercher en fonction de son nom
- si un résultat est retourné alors:
    - si plus d'un résultat, demander son choix à l'utilisateur
    - afficher la fiche
    - demander la confirmation
- si l'action a été confirmée alors:
    - script SQL: DELETE 

In [25]:
def supprimer_employe():
    
    
    """
    :name : supprimer_employe
    :param : 
    :return : l' argument de la fonction se_connecter
    :desc : supprimer la fiche d'un employé après l'avoir l'avoir cherchée et affichée
    """
    
    
    resultat = rechercher_employe()
    
    
    if resultat is not None:
   
        nb_res, last_res = resultat
        

        if nb_res != 1:
            last_res = input("Saisissez l'identifiant *** id_em *** de l'employé à supprimer: ")


        fiche_employe = afficher_employe(last_res)

        print(f"=====> Vous allez travailler sur la fiche {fiche_employe} ")
        confirmation = confirmer()


        if confirmation:
                     

            requete = f"DELETE FROM employes WHERE id_em = '{str(last_res)}' "

            print(requete)
            return requete
        
        
    
#supprimer_employe()



### Afficher une table

- Script SQL: SELECT
- imprimer avec fetchall()

In [26]:
def afficher_table(table):
    
    """
    :name : afficher_table
    :param : la table à afficher
    :return : l'affichage du résultat de la requête
    :desc : afficher simplement le contenu d'une table
    """
    
    
    try:
    
        curseur, connexion = connecter()

        requete = f"SELECT *  FROM {table}   "
        resultat = curseur.execute(requete)

        return print(curseur.fetchall())
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la table a échoué.")
        


## Afficher les statistiques

- Réponses aux 8 questions

Détail de la réponse à la question 3b: l'âge médian
- soit n le nombre de valeurs dans resulat, si n est paire:
    - alors je fais la moyenne de n/2 et n/2-1, par exemple 4/2=2 et 4/2-1=1, si j'ai 4 valeurs, je veux la moyenne de la 2ème et la 3ème valeur, donc les indexes 1 et 2 de resultat.
    - sinon je divise simplement n/2 recasté en integer pour que par exemple 3/2=1,5 et int(1.5)=1, ainsi l'indexe 1 est bien le 2ème élément de resultat

In [27]:
def afficher_stats():
    
    """
    :name : afficher_stats
    :param : 
    :return : 
    :desc : afficher l'ensemble des questions et leur réponse.
    """
    
    
    print("\n\n1- Combien de salariés comprend la société ?")
    print("Pour être précis, le nombre de salariés non sorti de l'effectif, ")
    print(f"     - l'effectif de la société  est {lire_reponse(question_1())[0][0]} employé(s).")
    
    
    print("\n\n2- Où sont-ils localisés géographiquement au niveau professionnel ?")
    reponse = lire_reponse(question_2())
    print(f"Parmi cet effectif, ")
    for num, r in enumerate (reponse):
        print(f"     - {reponse[num][1]} se trouve(nt) sur le site de {reponse[num][0]}.")
    
    
    print("\n\n3- Quel est l’âge moyen et médian des salariés ?")
    print(f"Parmi cet effectif, ")
    print(f"     - l'âge moyen est {lire_reponse(question_3a())[0][0]} ans.")
    reponse = lire_reponse(question_3b())
    if len(reponse)%2 == 0:
        print (f"     - l'âge médian qui partage l'effectif en 2 sous-effectifs égaux est {((reponse[int(len(reponse)/2)][0]) + (reponse[int(len(reponse)/2)-1][0]))/2} ans.")
    else:
        print(f"     - l'âge médian qui partage l'effectif en 2 sous-effectifs égaux est {reponse[int(len(reponse)/2)][0]} ans.")
    
    
    print("\n\n4- Quel est l’étendue des âges ?")
    print(f"Parmi cet effectif, ")
    reponse = lire_reponse(question_4())
    print(f"     - l'étendue des âges va de {lire_reponse(question_4())[0][0]} ans à {lire_reponse(question_4())[0][1]} ans ce qui donne une étendue de {lire_reponse(question_4())[0][2]} an(s).")
    
    
    print("\n\n5- Quel est le salarié le plus ancien de la société en termes d’ancienneté ?")
    reponse = lire_reponse(question_5())
    print(f"Parmi cet effectif, ")
    for num, r in enumerate (reponse):
        if reponse[num][4] == "Femme":
            civilite = "Madame"
        elif reponse[num][4] == "Homme":
            civilite = "Monsieur"
        else:
            civilite = "Autre"
        annee,mois,jour = reponse[num][2].split("-")
        print(f"     - c'est {civilite} {reponse[num][0]} {reponse[num][1]} né(e) le {jour}/{mois}/{annee} et qui travaille sur le site de {reponse[num][5]}")
        print(f"     - qui a la plus grande ancienneté avec {reponse[num][3]} ans.")
    
    
    print("\n\n6- Quelle est la répartition entre les hommes et les femmes ?")
    print(f"Parmi cet effectif, ")
    reponse = lire_reponse(question_6())
    total = 0
    for num, rep in enumerate(reponse):
        total += reponse[num][1]
    for num, rep in enumerate(reponse):
        print(f"     - {reponse[num][1]} sont des {reponse[num][0]}(s) soit {round(reponse[num][1] / total*100, 1)}% de l'effectif.")
    
    
    print("\n\n7- Y-a-t-il des doublons ?")
    print ("Calcul effectué sur le nom + le prénom + la date de naissance de l'effectif:")
    reponse = lire_reponse(question_7())
    if len(reponse) ==0:
        print ("     - il n'y a pas de doublon")
    else:
        print(f"     - les fiches potentiellement en doublon sont:\n {reponse}.")
    
    
    print("\n\n8- Afficher la liste des salariés classés par ordre alphabétique.")
    print ("Tri effectué sur le nom puis le prénom, de l'ensemble des employés :")
    reponse = lire_reponse(question_8())
    print("Liste des fiches,")
    print(f"     - la fiche n° x donne (id, nom, prenom, date_naissance, date_entree, date_sortie, genre, site).")
    for num, rep in enumerate(reponse):
        
        print(f"     - la fiche n° {num} donne {reponse[num]}.")
    

#afficher_stats()
    

In [28]:
def lire_reponse(resultats):
    
    """
    :name : lire_reponse
    :param : 
    :return : reponses
    :desc : afficher l'ensemble des réponses (ce qui sort du fetchall() est un NoneType, impossible à slicer)
    """
    
    reponses = []
    
    for resultat in resultats:
        
        reponses.append(resultat)
        
    return reponses
        

Exemple type du script SQL complet utilisé: 

- SELECT pour choisir les champs et faire des agrégats
    - FROM pour choisir la table
        - LEFT JOIN si besoin, pour lier des tables et récupérer leur champs (d'autres types sont possibles que LEFT et c'est là que la foreign key est pratique).
    - WHERE si besoin pour filter le résultat de la requête
    - GROUP BY si besoin pour le niveau d'agrégation
    - HAVING si besoin de filtrer sur un agrégat
    - ORDER BY pour trier le résultat sur les champs indiqués
                        

In [29]:
def question_1():
    
    """
    :name : question_1
    :param : 
    :return : le résultat de la requête
    :desc : afficher l'effectif de la société (employé non sorti des effectifs).
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT count(*)  
                        FROM employes 
                        WHERE date_sortie is null
                        
                        """
                       )
        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")



In [30]:
def question_2():
    
    """
    :name : question_2
    :param : 
    :return : le résultat de la requête
    :desc : afficher l'effectif de la société par site (employé non sorti des effectifs).
    """
    
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT site, count(*)  
                        FROM employes 
                            LEFT JOIN sites ON employes.id_si = sites.id_si
                        WHERE date_sortie is null
                        GROUP BY site
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")



Complément d'information sur le script SQL: 

- julianday pour mieux extraire la date et donc être plus précis
- AVG pour faire la moyenne
- ROUND pour arrondir le résultat à 1 décimale

In [31]:
def question_3a():
    
    """
    :name : question_3a
    :param : 
    :return : le résultat de la requête
    :desc : afficher l'âge moyen des employés de la société (employé non sorti des effectifs).
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT ROUND(AVG((julianday(date()) - julianday(date_naissance))/365.25),1) as age_moy
                        FROM employes 
                        WHERE date_sortie is null
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        


Complément d'information sur le script SQL: 

- julianday pour mieux extraire la date et donc être plus précis
- ROUND pour arrondir le résultat à 1 décimale
- ORDER BY pour trier

Le calcul de la médianne étant réalisé dans afficher_stats, je lui fourni une liste des âges dans l'ordre croissant

In [32]:
def question_3b():
    
    """
    :name : question_3b
    :param : 
    :return : le résultat de la requête
    :desc : lister les âges des employés de la société (employé non sorti des effectifs) 
            pour réaliser le calcul de la médianne
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT ROUND(((julianday(date()) - julianday(date_naissance))/365.25),1) as age
                        FROM employes 
                        
                        WHERE date_sortie is null
                        
                        ORDER BY age
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        


Complément d'information sur le script SQL: 

- julianday pour mieux extraire la date et donc être plus précis
- MIN pour avoir la valeur minimale
- MAX pour avoirla valeur maximale
- ROUND pour arrondir le résultat à 1 décimale

In [33]:
def question_4():
    
    """
    :name : question_4
    :param : 
    :return : le résultat de la requête
    :desc : calculer l'étendue des âges des employés de la société (employé non sorti des effectifs)
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT  
                        ROUND(MIN((julianday(date()) - julianday(date_naissance))/365.25),1) as age_min, 
                        ROUND(MAX((julianday(date()) - julianday(date_naissance))/365.25),1) as age_max,
                        ROUND((MAX((julianday(date()) - julianday(date_naissance))/365.25) - 
                               MIN((julianday(date()) - julianday(date_naissance))/365.25)),1) as age_etendue
                        FROM employes 
                        WHERE date_sortie is null
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        

        

Complément d'information sur le script SQL: 
- le but est quand-même d'afficher la fiche de l'employé
- donc ajout d'une sous-requête dans la clause WHERE pour servir de condition

nb: pas impossible d'avoir plus d'un employé.

In [34]:
def question_5():
    
    """
    :name : question_5
    :param : 
    :return : le résultat de la requête
    :desc : trouver l'employés de la société qui a la plus grande ancienneté (employé non sorti des effectifs)
    """
        
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT nom, prenom, date_entree, 
                               ROUND(((julianday(date()) - julianday(date_entree))/365.25),1) as anciennete, genre, site
                        FROM employes 
                            LEFT JOIN genres ON employes.id_ge = genres.id_ge
                            LEFT JOIN sites ON employes.id_si = sites.id_si
                        WHERE date_sortie is null
                            AND anciennete = (SELECT MAX(ROUND(((julianday(date()) - julianday(date_entree))/365.25),1))
                                              FROM employes
                                              WHERE date_sortie is null
                                              )
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        

#question_5()

In [35]:
def question_6():
    
    """
    :name : question_6
    :param : 
    :return : le résultat de la requête
    :desc : trouver l'effectif par genre de la société (employé non sorti des effectifs)
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT genre, count(*)
                        FROM employes 
                            LEFT JOIN genres ON employes.id_ge = genres.id_ge
                            
                        WHERE date_sortie is null
                        GROUP BY genre
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        


Complément d'information sur le script SQL: 
- Compter le nombre de fiche ayant le nom + prenom + date_naissance en commun
- n'afficher que celles dont le nombre est supérieur à 1

In [36]:
def question_7():
    
    """
    :name : question_7
    :param : 
    :return : le résultat de la requête
    :desc : vérifier qu'il n'y a pas de doublon dans la société (employé non sorti des effectifs)
            est comparer le nom + prenom + date_naissance
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT nom, prenom, date_naissance, count(*)
                        FROM employes
                        WHERE date_sortie is null
                        GROUP BY nom, prenom, date_naissance
                        HAVING count(*) > 1
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        


In [37]:
def question_8():
    
    """
    :name : question_8
    :param : 
    :return : le résultat de la requête
    :desc : afficher la liste de tous les employés de la société
    """
    
    try:
    
        curseur, connexion = connecter()

        curseur.execute("""
                        SELECT id_em, nom, prenom, date_naissance, date_entree, date_sortie, genre, site
                        FROM employes 
                            LEFT JOIN genres ON employes.id_ge = genres.id_ge
                            LEFT JOIN sites ON employes.id_si = sites.id_si
                        
                        ORDER BY nom, prenom
                        
                        
                        """
                       )

        resultat = curseur.fetchall()
        return resultat
    
        deconnecter(curseur,connexion)
    
    except sqlite3.Error as e:
        print(e,"\n\n=====> L'affichage de la réponse a échoué.")
        


# MENUS
============

- 3 menus pour faciliter la navigation
- 1 fonction pour la sélection des choix
- 1 fonction pour confirmer les choix

## Menu général

- Afficher la liste des choix
- conditions sur les choix


In [38]:


def menu_general():
    print("*************************************************************************")
    print("*                         MENU GENERAL                                  *")
    print("*                     Gestion_personnel.db                              *")
    print("*                                                                       *")
    print("*************************************************************************")

    afficher_liste = ["menu administrateur","menu utilisateur","quitter la base"]
    choix = menu_selection(afficher_liste)
    
    if choix == 0:
        
        password_requis = "pagayons"
        password_donne = input("Cet accès nécessite un mot de passe: ")
        if password_donne == password_requis:
            print("\n\n=====> Juste: accès au menu administrateur")
            menu_administrateur()
        else:
            print("\n\n=====> Faux: retour au menu general ")
            menu_general()

    elif choix == 1:
        
        menu_utilisateur()
    
    elif choix == 2:
        
        print("*************************************************************************")
        print("*                         FIN DE SESSION                                *")
        print("*                                                                       *")
        print("*************************************************************************")



## Menu administrateur

- Créer la liste des choix
- lancer la fonction menu_selection
- conditions sur les choix


In [39]:
def menu_administrateur():
        
    """
    :name : menu_administrateur
    :param : 
    :return : 
    :desc : permettre à l'administrateur de naviguer dans un menu
    """
    
    print("*************************************************************************")
    print("*                     MENU ADMINISTRATEUR                               *")
    print("*                     Gestion_personnel.db                              *")
    print("*                                                                       *")
    print("*************************************************************************")
    
    afficher_liste = ["créer la base et les tables","charger les tables"]
    choix = menu_selection(afficher_liste)
    
    if choix == 0:
        print("La base de données et ses 3 tables vont être créées.")
        confirmation = confirmer()
        if confirmation:

            curseur, connexion = connecter()
            creer_employes(curseur)
            creer_sites(curseur)
            creer_genres(curseur)
            print("La base de données et ses 3 tables (+1) ont été créées:")
            print(verifier(curseur))
                
                
            verifier_date_entree(curseur)
            verifier_date_sortie(curseur)
            deconnecter(curseur,connexion)
             
            curseur, connexion = connecter()
            foreign_key_contrainst_genre(curseur)
            foreign_key_contrainst_site(curseur)
            
            print("4 triggers ont été ajoutés")
                
            deconnecter(curseur,connexion)
            
        print("\n\n=====>Terminé: retour au menu administrateur")
        menu_administrateur()


    elif choix == 1:
            

        curseur, connexion = connecter()
        tuple_tables = verifier(curseur)
        if len(tuple_tables) != 4:
            print("Vous n'avez pas créé la base de données et ses 3 tables")
            deconnecter(curseur, connexion)    

        else:

            print("Vous allez charger les 3 tables une par une.")
            confirmation = confirmer()
            if confirmation:

                print("Etes-vous prêt à charger la table employés via la liste_employes.csv?")
                confirmation = confirmer()
                if confirmation:
                    curseur, connexion = connecter()
                    print(charger_employes(curseur,connexion))
                    deconnecter(curseur,connexion)


                print("Etes-vous prêt à charger la table sites par défaut")
                confirmation = confirmer()
                if confirmation:
                    curseur, connexion = connecter()
                    print(charger_sites(curseur,connexion))
                    deconnecter(curseur,connexion)


                print("Etes-vous prêt à charger la table genres par défaut")
                confirmation = confirmer()
                if confirmation:
                    curseur, connexion = connecter()
                    print(charger_genres(curseur, connexion))
                    deconnecter(curseur,connexion)
    
        print("\n\n=====>Terminé: retour au menu administrateur")
        menu_administrateur()
    
 
    
#menu_administrateur()

## Menu utilisateur

- Créer la liste des choix
- lancer la fonction menu_selection
- conditions sur les choix


In [40]:
def menu_utilisateur():
    
    """
    :name : menu_utilisateur
    :param : 
    :return : 
    :desc : permettre à l'utilisateur de naviguer dans un menu
    """
    
    print("*************************************************************************")
    print("*                       MENU UTILISATEUR                                *")
    print("*                     Gestion_personnel.db                              *")
    print("*                                                                       *")
    print("*************************************************************************")
    
    
    afficher_liste = ["travailler sur une table","afficher les statistiques"]
    choix = menu_selection(afficher_liste)
    
 
    
    if choix == 0:
        
        travailler_tables()
    elif choix == 1:
        
        afficher_stats()
        print("\n\n=====>Terminé: retour au menu utilisateur")
        menu_utilisateur()
        

#menu_utilisateur()

## Selection des choix

- afficher la liste des choix
- demander son choix à l'utilisateur
- condition sur son choix:
    - si invalide alors retour au menu général
    - si valide alors continuer

In [41]:
def menu_selection(afficher_liste):
    
    """
    :name : menu_selection
    :param : liste des choix issue des fonctions menu_utilisateur ou menu_administrateur
    :return : le choix de l'utilisteur
    :desc : permettre à l'utilisateur de sélectionner son choix
    """
    
    
    print("Voici les choix disponibles:")
    for num,liste in enumerate (afficher_liste):
        print(num, ": ", afficher_liste[num])
    travail_liste = int(input("quel est votre choix? (un autre code revient au menu général)"))
    
    if travail_liste >= len(afficher_liste) :
        print(f"\n\n=====> {travail_liste}: retour au menu général")
        menu_general()
    else:
        print(f"\n=====> {travail_liste}: accès à {afficher_liste[travail_liste]}")
    return travail_liste

## Confirmation des actions

- Demander si on confirme
- si vrai alors on exécute
- si faux alors on n'exécute pas.
- retouner un booléen pour la condition dans la fonction source

In [42]:
def confirmer():
    
    """
    :name : confirmer
    :param : 
    :return : la réponse de l'utilisateur
    :desc : permettre à l'utilisateur de cpnfirmer ses actions
    """
        
    
    confirmation = input("Confirmez-vous? (o/n)")
    if confirmation.lower() == "o":
        print(f"\n=====> {confirmation}: l'action sera exécutée")
    else:
        print(f"=====> {confirmation}: l'action ne sera pas exécutée")
    return confirmation.lower() == "o"

# LANCER LE PROGRAMME
=========================================


In [43]:
menu_general()

*************************************************************************
*                         MENU GENERAL                                  *
*                     Gestion_personnel.db                              *
*                                                                       *
*************************************************************************
Voici les choix disponibles:
0 :  menu administrateur
1 :  menu utilisateur
2 :  quitter la base

=====> 1: accès à menu utilisateur
*************************************************************************
*                       MENU UTILISATEUR                                *
*                     Gestion_personnel.db                              *
*                                                                       *
*************************************************************************
Voici les choix disponibles:
0 :  travailler sur une table
1 :  afficher les statistiques

=====> 1: accès à afficher les stati